### LLM

**LangSmith 추적**

In [80]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

gpt_4o_mini = ChatOpenAI(temperature=0, 
                    model_name="gpt-4o-mini",
                    streaming=True,              
                    callbacks=[StreamingStdOutCallbackHandler()]
                    )

gemini_1_5_flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# Mongodb 

In [81]:
from pymongo import MongoClient 
import os

class MongoDB:
    def __init__(self, collection_name: str):
        # Set up MongoDB connection using environment variables
        client = MongoClient(os.getenv("MONGODB_URI"))
        db = client[os.getenv("MONGODB_DB_NAME")]
        self.collection = db[collection_name]

    def find_one_booking_reference(self, booking_reference):
        return self.collection.find_one({'bookingReference': booking_reference})

# Langgraph

### State 정의

In [82]:
from typing import TypedDict

class State(TypedDict):
    booking_reference: str
    bkg_data: dict
    si_data: dict
    missing_answer: str
    summary_answer: str
    next: str

### Langchain functions

**get SI or BKG data**

In [83]:
class GetBKG:
    def __init__(self):
        self.mongodb = MongoDB(collection_name="bkg")
    
    def __call__(self, state: State) -> State:
        bkg_data = self.mongodb.find_one_booking_reference(state["booking_reference"])
        
        if bkg_data is None:
            state["bkg_data"] = "No Booking Data found for the given booking reference"
            state["next"] = "end"
        else:
            state["bkg_data"] = bkg_data
            state["next"] = "get_si"
        return state

class GetSI:
    def __init__(self):
        self.mongodb = MongoDB(collection_name="si")
    
    def __call__(self, state: State) -> State:
        si_data = self.mongodb.find_one_booking_reference(state["booking_reference"])
        if si_data is None:
            state["si_data"] = "No Shipping Instruction found for the given booking reference"
            state["next"] = "end"
        else:
            state["si_data"] = si_data
            state["next"] = "check_missing_data"
        return state

### Check Missing data

In [84]:
from langchain_core.output_parsers import JsonOutputParser
from common.schemas import ShipmentStatus
from langchain.prompts import PromptTemplate
from common.prompts import check_missing_prompt



class CheckMissingData:
    def __init__(self):
        self.output_parser = JsonOutputParser(pydantic_object=ShipmentStatus)
        #output_parser.parse() 메서드를 사용하여 JSON 출력을 파싱
        #ShipmentStatus Pydantic 모델에 따라 유효성 검사를 수행하고, ShipmentStatus 객체 생성
        
        self.llm = gpt_4o_mini
        self.prompt = PromptTemplate(
            template=check_missing_prompt, 
            input_variables=["si_data"], #동적으로 삽입될 변수
            partial_variables={ 
                "format_instructions": self.output_parser.get_format_instructions()
                #partial_variables는 PromptTemplate을 생성할 때 프롬프트 템플릿에 미리 정의된 값을 삽입하는 역할
            },
        )
        self.chain = self.prompt | self.llm | self.output_parser
    
    def __call__(self, state: State) -> State:
        try:
            # Synchronously invoke the LLM to check for missing data
            response = self.chain.invoke({"si_data": state["si_data"]})
            state['missing_answer'] = response 
        except Exception as e:
            state['missing_answer'] = f"Error during checking missing data: {e}"
        
        state['next'] = "generate_intake_report"
        return state

In [85]:
from common.schemas import ShipmentSummary
from common.prompts import intake_report_prompt

class GenerateIntakeReport:
    def __init__(self):
        self.output_parser = JsonOutputParser(pydantic_object=ShipmentSummary)
        self.llm = gpt_4o_mini
        self.prompt = PromptTemplate(
            template=intake_report_prompt,
            input_variables=["si_data", "missing_info"],
            partial_variables={
                "format_instructions": self.output_parser.get_format_instructions()
            },
        )
        self.chain = self.prompt | self.llm | self.output_parser

    def __call__(self, state: State) -> State:
        try:
            response = self.chain.invoke(
                {   
                    "si_data" : state['si_data'],
                    "missing_info": state["missing_answer"],
                }
            )
            state['summary_answer'] = response
        except Exception as e:
            state['summary_answer'] = f"Error generating summary: {e}"
        return state

### graph 연결

In [86]:
from langgraph.graph import StateGraph, END

class SIIntake:
    def __init__(self):
        self.get_bkg_node = GetBKG()
        self.get_si_node = GetSI()
        self.check_missing_data_node = CheckMissingData()
        self.generate_intake_report_node = GenerateIntakeReport()
        self.graph = self.generate_graph()
        self.state = State()

    def generate_graph(self):
        workflow = StateGraph(State)

        # Add nodes
        workflow.add_node("get_bkg", self.get_bkg_node)
        workflow.add_node("get_si", self.get_si_node)
        workflow.add_node("check_missing_data", self.check_missing_data_node)
        workflow.add_node("generate_intake_report", self.generate_intake_report_node)

        # Add edges
        workflow.set_entry_point("get_bkg")
        workflow.add_conditional_edges(
            "get_bkg", 
            lambda state: state['next'],
            {
                "get_si": "get_si", 
                "end": END
            }
        )
        workflow.add_conditional_edges(
            "get_si", 
            lambda state: state['next'],
            {
                "check_missing_data": "check_missing_data", 
                "end": END
            }
        )
        workflow.add_edge("check_missing_data", "generate_intake_report")
        workflow.add_edge("generate_intake_report", END)

        return workflow.compile()
    
    def invoke(self):
        return self.graph.invoke(self.state)

In [87]:
graph = SIIntake()
graph.state["booking_reference"] = "CHERRY202409072244"
graph.invoke()

```json
{
  "vessel_route_details": {
    "vessel_name": {
      "status": "OK"
    },
    "voyage_number": {
      "status": "OK"
    },
    "place_of_receipt": {
      "status": "OK"
    },
    "port_of_loading": {
      "status": "OK"
    },
    "port_of_discharge": {
      "status": "OK"
    },
    "place_of_delivery": {
      "status": "OK"
    },
    "total_status": "OK"
  },
  "payment_documentation": {
    "freight_payment_terms": {
      "status": "OK"
    },
    "bl_type": {
      "status": "OK"
    },
    "number_of_original_bls": {
      "status": "OK"
    },
    "total_status": "OK"
  },
  "party_information": {
    "status": {
      "status": "OK"
    },
    "total_status": "OK"
  },
  "shipping_details": {
    "status": {
      "status": "OK"
    },
    "total_status": "OK"
  },
  "container_information": {
    "status": {
      "status": "OK"
    },
    "total_status": "OK"
  },
  "total_shipment_summary": {
    "status": {
      "status": "OK"
    },
    "total_status"

{'booking_reference': 'CHERRY202409072244',
 'bkg_data': {'_id': ObjectId('66ff7f2352ff68133756ca2d'),
  'bookingReference': 'CHERRY202409072244',
  'customerName': 'CUSTOMER 9946',
  'shipperName': 'SHIPPER 5029',
  'invoiceReceiver': 'INVOICE RECEIVER 3204',
  'voyageDetails': {'vesselName': 'APL TEMASEK', 'voyageNumber': '2024581E'},
  'cargoDetails': {'hsCode': '132054',
   'chapterDescription': 'SAMPLE CHAPTER DESCRIPTION 31',
   'commodity': 'SAMPLE COMMODITY 89'},
  'containerDetails': {'size': '45 HIGH CUBE',
   'type': 'REEFER CONTAINER',
   'quantity': 4},
  'routeDetails': {'placeOfReceipt': 'NINGBO',
   'portOfLoading': 'NINGBO',
   'portOfDischarge': 'YANTIAN',
   'placeOfDelivery': 'YANTIAN'},
  'scheduleDetails': {'estimatedArrivalAtLoadingPort': '2024-09-14 21:26',
   'estimatedDepartureFromLoadingPort': '2024-10-30 21:26',
   'estimatedArrivalAtDischargePort': '2024-11-17 21:26'},
  'emptyContainerPickupLocation': 'LE HAVRE, JAPAN',
  'shippingTerm': 'DOOR/DOOR',
  're

In [88]:
# 환경 변수 목록 가져오기
env_vars = dict(os.environ)

# 환경 변수 목록 출력
for key, value in env_vars.items():
    print(f"{key}: {value}")

COMMAND_MODE: unix2003
CONDA_EXE: /opt/homebrew/anaconda3/bin/conda
CONDA_PYTHON_EXE: /opt/homebrew/anaconda3/bin/python
CONDA_SHLVL: 1
HOME: /Users/rohhail
HOMEBREW_CELLAR: /opt/homebrew/Cellar
HOMEBREW_PREFIX: /opt/homebrew
HOMEBREW_REPOSITORY: /opt/homebrew
INFOPATH: /opt/homebrew/share/info:
LESS: -R
LOGNAME: rohhail
LSCOLORS: Gxfxcxdxbxegedabagacad
LS_COLORS: di=1;36:ln=35:so=32:pi=33:ex=31:bd=34;46:cd=34;43:su=30;41:sg=30;46:tw=30;42:ow=30;43
MallocNanoZone: 0
OLDPWD: /
ORIGINAL_XDG_CURRENT_DESKTOP: undefined
P9K_SSH: 0
PAGER: cat
PATH: /opt/homebrew/anaconda3/envs/sigenie/bin:/opt/homebrew/opt/openjdk/bin:/opt/homebrew/anaconda3/condabin:/Library/Frameworks/Python.framework/Versions/3.12/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/c